In [15]:
# 필요한 패키지 설치: pip install sxtwl pytz
import sxtwl
from datetime import datetime
from pytz import timezone

# 십간, 십이지 리스트
GAN = ["갑", "을", "병", "정", "무", "기", "경", "신", "임", "계"]
ZHI = ["자", "축", "인", "묘", "진", "사", "오", "미", "신", "유", "술", "해"]

def get_ganzhi(gz):
    return GAN[gz.tg] + ZHI[gz.dz]

def get_sexagenary_info(birth_str, birth_time_str, gender="남자", is_lunar=False, is_leap=False):
    """
    생년월일과 시간 기준으로 사주 및 운세 출력
    - birth_str: "YYYYMMDD"
    - birth_time_str: "HHMM" or "모름"
    - gender: "남자" or "여자"
    - is_lunar: 음력 입력 여부
    - is_leap: 윤달 여부 (음력인 경우)
    """
    # 1. 입력 날짜 및 시간 처리
    birth_date = datetime.strptime(birth_str, "%Y%m%d")
    if birth_time_str != "모름":
        birth_time = datetime.strptime(birth_time_str, "%H%M").time()
        birth_date = datetime.combine(birth_date, birth_time)
    else:
        birth_date = datetime.combine(birth_date, datetime.min.time())

    # 2. 한국 시간대 적용
    birth_date = timezone("Asia/Seoul").localize(birth_date)
    y, m, d = birth_date.year, birth_date.month, birth_date.day

    # 3. 간지 계산용 날짜 객체 생성
    if not is_lunar:
        day = sxtwl.fromSolar(y, m, d)
    else:
        day = sxtwl.fromLunar(y, m, d, is_leap)

    # 4. 년주, 월주, 일주
    year_gz = day.getYearGZ(True)   # 입춘 기준
    month_gz = day.getMonthGZ()
    day_gz = day.getDayGZ()

    # 5. 시주 계산
    if birth_time_str != "모름":
        hour = birth_date.hour
        branch_index = ((hour + 1) // 2) % 12
        stem_index = (day_gz.tg * 2 + branch_index) % 10
        hour_gz = GAN[stem_index] + ZHI[branch_index]
    else:
        hour_gz = "모름"

    # 6. 현재 시점 기준 세운, 월운
    now = datetime.now(timezone("Asia/Seoul"))
    now_day = sxtwl.fromSolar(now.year, now.month, now.day)
    current_year_gz = get_ganzhi(now_day.getYearGZ(True))
    current_month_gz = get_ganzhi(now_day.getMonthGZ())

    # 7. 대운 계산 (단순화된 버전)
    is_male = gender == "남자"
    is_yang = year_gz.tg % 2 == 0
    forward = (is_male and is_yang) or (not is_male and not is_yang)

    base_index = (month_gz.tg * 10 + month_gz.dz) % 60
    current_age = now.year - y
    steps = current_age // 10

    if forward:
        du_index = (base_index + steps) % 60
    else:
        du_index = (base_index - steps) % 60

    du_gan = GAN[du_index % 10]
    du_zhi = ZHI[du_index % 12]
    current_daewoon = du_gan + du_zhi

    # 8. 결과 반환
    return {
        "년주": get_ganzhi(year_gz),
        "월주": get_ganzhi(month_gz),
        "일주": get_ganzhi(day_gz),
        "시주": hour_gz,
        # "대운": current_daewoon,
        # "세운": current_year_gz,
        # "월운": current_month_gz
    }

# 사용 예시
result = get_sexagenary_info("19970314", "0945", gender="남자", is_lunar=False)
# for k, v in result.items():
#     print(f"{k}: {v}")
result

{'년주': '정축', '월주': '계묘', '일주': '을묘', '시주': '신사'}

In [36]:
from openai import OpenAI
import json

client = OpenAI()

ilju = "기사"
mbit = "INTP"

system_prompt = "당신은 사주와 MBTI를 통해 사람의 성격을 분석하는 전문가입니다."
user_prompt = f"사주는 {ilju}일주이고, MBTI는 {mbit}입니다. 이 사람의 성격을 분석하여 형태에 맞게 출력하세요."

tools = [{
    "type": "function",
    "name": "get_analysis",
    "description": "사주와 MBTI를 통해 사람의 성격을 분석합니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "외향성": {
                "type": "number",
                "description": "외향성 점수를 1~5 사이의 정수로 평가합니다."
            },
            "안정성": {
                "type": "number", 
                "description": "안정성 점수를 1~5 사이의 정수로 평가합니다."
            },
            "개방성": {
                "type": "number",
                "description": "개방성 점수를 1~5 사이의 정수로 평가합니다."
            },
            "책임감": {
                "type": "number",
                "description": "책임감 점수를 1~5 사이의 정수로 평가합니다."
            },
            "친화성": {
                "type": "number",
                "description": "친화성 점수를 1~5 사이의 정수로 평가합니다."
            },
            "자기주도성": {
                "type": "number",
                "description": "자기주도성 점수를 1~5 사이의 정수로 평가합니다."
            },
            "이성적": {
                "type": "number",
                "description": "이성적 점수를 1~5 사이의 정수로 평가합니다."
            },
            "도전성": {
                "type": "number",
                "description": "도전성 점수를 1~5 사이의 정수로 평가합니다."
            },
            "MBIT기반 직업": {
                "type": "string",
                "description": "MBIT기반 직업을 세 가지 추천합니다. 예시: 브랜드 마케터, 조직문화 코치, 교육 기획자"
            },
            "사주기반 직업": {
                "type": "string",
                "description": "사주기반 직업을 세 가지 추천합니다. 예시: 스토리 기반 콘텐츠 기획자, 식물·자연·감성 기반 디자인 전문가, 문화·예술 분야의 중재자"
            },
            "최종 직업": {
                "type": "string",
                "description": "MBIT기반 직업과 사주기반 직업을 참고하여 최종 직업을 한 문장으로 나타냅니다. 예시: 사람의 감정을 기획으로 연결하는 공감형 BX 디자이너"
            },
            "성격 키워드": {
                "type": "string",
                "description": "MBIT와 사주를 기반으로 성격 키워드를 세 가지 태그로 나타냅니다. 예시: #섬세한 #사람을이끄는 #공감형"
            }
        },
        "required": [
            "외향성", "안정성", "개방성", "책임감", "친화성", "자기주도성", "이성적", "도전성",
            "MBIT기반 직업", "사주기반 직업", "최종 직업", "성격 키워드"
        ],
        "additionalProperties": False
    }
}]

response = client.responses.create(
    model="gpt-4.1",
    input=[{"role": "system", "content": system_prompt},
           {"role": "user", "content": user_prompt}],
    tools=tools
)

arguments = json.loads(response.output[0].arguments)
print(arguments)

{'외향성': 2, '안정성': 4, '개방성': 5, '책임감': 3, '친화성': 2, '자기주도성': 4, '이성적': 5, '도전성': 4, 'MBIT기반 직업': '문제 해결을 필요로 하는 연구원, 데이터 분석가, 시스템 설계자', '사주기반 직업': '논리적 사고 기반 컨설턴트, IT·기술 분야 전문가, 전략 기획자', '최종 직업': '창의적 해결책을 제시하는 전략적 IT 컨설턴트', '성격 키워드': '#분석적 #독립적 #창의적'}
